In [196]:
import sys
import os
# caution: path[0] is reserved for script path (or '' in REPL).
sys.path.insert(1, os.path.abspath('./../src'))

import datetime
import numpy as np
import psd_tool
import data_loader
from field_models import model
import matplotlib.pyplot as plt

SMALL_SIZE = 12
MEDIUM_SIZE = 18
BIGGER_SIZE = 20

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=BIGGER_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=8)    # legend fontsize
plt.rc('figure', titlesize=MEDIUM_SIZE)  # fontsize of the figure title

import importlib
importlib.reload(psd_tool)
importlib.reload(data_loader)

%matplotlib qt
plt.ioff() #This is awesome and delays the drawing until plt.show(). Thank god



In [205]:
#Data Properties
start = datetime.datetime(year = 2013,
                            month = 3,
                            day = 5) - datetime.timedelta(hours=8.5)

end = datetime.datetime(year = 2013, 
                        month = 3, 
                        day = 16) - datetime.timedelta(hours=8.5)

satellite = "A"
chosen_mu = 1000
chosen_k = 0.05
model = model.TS04D

refs = data_loader.load_psd(satellite=satellite, field_model=model, start=start, end=end)

selected_refs = psd_tool.select_mu_and_k_from_psd(refs=refs, chosen_mu=chosen_mu, chosen_k=chosen_k)

epoch = selected_refs["EPOCH"]
Lstar = selected_refs["L_STAR"] 
L = selected_refs["L"]
PSD = selected_refs["PSD"]
in_out = selected_refs["IN_OUT"]
orbit_number = selected_refs["ORBIT_NUMBER"]

not_nan = np.isfinite(PSD)
epoch = epoch[not_nan]
Lstar = Lstar[not_nan]
L = L[not_nan]
PSD = PSD[not_nan]
in_out = in_out[not_nan]
orbit_number = orbit_number[not_nan]

2013-03-01 00:00:00
Loading : PSD_201303_A_TS04D.npz
Time taken for loop: 5.774267673492432


In [206]:
#Simulation Properties
min_lstar = 2.0
max_lstar = 6.0
dL = 0.05


In [207]:
fig, axs = plt.subplots(1,2, sharex=True, sharey=True)
fig.set_size_inches(14, 10)

#Leaving out the first orbit should get us the full orbit that we started on..
unique_orbits = np.sort(np.unique(orbit_number))[1:]
start_of_orbits = []

for orbit in unique_orbits:
    
    inbound = (orbit_number == orbit) & (in_out == -1)
    outbound = (orbit_number == orbit) & (in_out == 1)
    
    if np.any(np.nonzero(inbound)[0]):
        
        start_of_inbound_orbit = epoch[np.nonzero(inbound)[0][0]]
        
        binned_Lstar, binned_PSD = psd_tool.bin_radial_profile(LSTAR = Lstar[inbound],
                                                               PSD = PSD[inbound],
                                                               LSTAR_MIN = min_lstar,
                                                               LSTAR_MAX = max_lstar,
                                                               dL = dL)
        
        psd_tool.plot_radial_profile(LSTAR = binned_Lstar,
                                     PSD = binned_PSD,
                                     IS_INBOUND = True,
                                     START_OF_ORBIT = start_of_inbound_orbit,
                                     AXIS = axs[0])
        
        start_of_orbits.append(start_of_inbound_orbit)

    if np.any(np.nonzero(outbound)[0]):
        
        start_of_outbound_orbit = epoch[np.nonzero(outbound)[0][0]]
        
        binned_Lstar, binned_PSD = psd_tool.bin_radial_profile(LSTAR = Lstar[outbound],
                                                               PSD = PSD[outbound],
                                                               LSTAR_MIN = min_lstar,
                                                               LSTAR_MAX = max_lstar,
                                                               dL = dL)
        
        psd_tool.plot_radial_profile(LSTAR = binned_Lstar,
                                     PSD = binned_PSD,
                                     IS_INBOUND = False,
                                     START_OF_ORBIT = start_of_outbound_orbit,
                                     AXIS = axs[0])
        
        start_of_orbits.append(start_of_outbound_orbit)

axs[0].set_axisbelow(True)
axs[0].yaxis.grid(color='gray', linestyle='dashed')
axs[0].xaxis.grid(color='gray', linestyle='dashed')
axs[0].set_xlabel("L*")
axs[0].set_ylabel("PSD $(c/MeV/cm)^{3}$")
axs[0].set_xlim((min_lstar, max_lstar))

handles, labels = axs[0].get_legend_handles_labels()
order = np.argsort(start_of_orbits)
colors = plt.get_cmap("viridis", len(order)) # Initialize holder for trajectories
j = 0
for i in order:
    handles[i].set_color(colors(j))
    j += 1

axs[0].legend([handles[i] for i in order], [labels[i] for i in order])
axs[0].set_title(rf"RBSP-{satellite.upper()}, $\mu$ = {chosen_mu} (MeV / G), K = {chosen_k} ($R_E$$G^{{1/2}}$), {model.name}")

Text(0.5, 1.0, 'RBSP-A, $\\mu$ = 1000 (MeV / G), K = 0.05 ($R_E$$G^{1/2}$), TS04D')

In [208]:
#Calculate which half orbit is first, this will be our initial condition..

initial_orbit_number = unique_orbits[0]
initial_orbit = (orbit_number == initial_orbit_number)
initial_in_out = in_out[initial_orbit][0]
initial_half_orbit = (orbit_number == initial_orbit_number) & (in_out == 1)
initial_PSD = PSD[initial_half_orbit]
initial_Lstar = Lstar[initial_half_orbit]
initial_epoch = epoch[initial_half_orbit]

binned_initial_Lstar, binned_initial_PSD = psd_tool.bin_radial_profile(LSTAR = initial_Lstar,
                                                       PSD = initial_PSD,
                                                       LSTAR_MIN = min_lstar,
                                                       LSTAR_MAX = max_lstar,
                                                       dL = dL)

psd_tool.plot_radial_profile(LSTAR = binned_initial_Lstar,
                             PSD = binned_initial_PSD,
                             IS_INBOUND = False,
                             START_OF_ORBIT = initial_epoch[0],
                             AXIS = axs[1])

axs[1].set_axisbelow(True)
axs[1].yaxis.grid(color='gray', linestyle='dashed')
axs[1].xaxis.grid(color='gray', linestyle='dashed')
axs[1].set_xlabel("L*")
axs[1].set_ylabel("PSD $(c/MeV/cm)^{3}$")
axs[1].set_xlim((min_lstar, max_lstar))

axs[1].set_title(rf"1-D Diffusion, RBSP-{satellite.upper()}, $\mu$ = {chosen_mu} (MeV / G), K = {chosen_k} ($R_E$$G^{{1/2}}$), {model.name}")
plt.show()